IPL Predictor

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# Read dataset
df_matches = pd.read_csv('matches.csv',header=0)

In [3]:
# brief info
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 18 columns):
id                 577 non-null int64
season             577 non-null int64
city               570 non-null object
date               577 non-null object
team1              577 non-null object
team2              577 non-null object
toss_winner        577 non-null object
toss_decision      577 non-null object
result             577 non-null object
dl_applied         577 non-null int64
winner             574 non-null object
win_by_runs        577 non-null int64
win_by_wickets     577 non-null int64
player_of_match    574 non-null object
venue              577 non-null object
umpire1            577 non-null object
umpire2            577 non-null object
umpire3            0 non-null float64
dtypes: float64(1), int64(5), object(12)
memory usage: 81.2+ KB


In [4]:
# First Five Entries
df_matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2008,Bangalore,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Kolkata Knight Riders,140,0,BB McCullum,M Chinnaswamy Stadium,Asad Rauf,RE Koertzen,NaN
1,2,2008,Chandigarh,2008-04-19,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,normal,0,Chennai Super Kings,33,0,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",MR Benson,SL Shastri,NaN
2,3,2008,Delhi,2008-04-19,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,normal,0,Delhi Daredevils,0,9,MF Maharoof,Feroz Shah Kotla,Aleem Dar,GA Pratapkumar,NaN
3,4,2008,Mumbai,2008-04-20,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,normal,0,Royal Challengers Bangalore,0,5,MV Boucher,Wankhede Stadium,SJ Davis,DJ Harper,NaN
4,5,2008,Kolkata,2008-04-20,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,normal,0,Kolkata Knight Riders,0,5,DJ Hussey,Eden Gardens,BF Bowden,K Hariharan,NaN


In [5]:
# dropping season , date , player of match , umpire 1,2,3
df_matches.drop(['season','date','player_of_match','umpire1','umpire2','umpire3'],axis = 1,inplace = True)
df_matches.head()

,id,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,venue
0,1,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Kolkata Knight Riders,140,0,M Chinnaswamy Stadium
1,2,Chandigarh,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,normal,0,Chennai Super Kings,33,0,"Punjab Cricket Association Stadium, Mohali"
2,3,Delhi,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,normal,0,Delhi Daredevils,0,9,Feroz Shah Kotla
3,4,Mumbai,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,normal,0,Royal Challengers Bangalore,0,5,Wankhede Stadium
4,5,Kolkata,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,normal,0,Kolkata Knight Riders,0,5,Eden Gardens


In [6]:
# drop na entries
df_matches.dropna(axis = 0,how = 'any',inplace = True)

In [7]:
# Dropping if any values are not available
df_matches.loc[(df_matches['team1'] != df_matches['toss_winner'])&(df_matches['team2'] != df_matches['toss_winner'])]
df_matches.loc[(df_matches['team1'] != df_matches['winner'])&(df_matches['team2'] != df_matches['winner'])]
# No ambiguitiy in winner and toss winner

,id,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,venue


In [8]:
#  Now giving team codes to every team in team 1 and team 2

teams1 = df_matches.team1.unique()

teams2 = df_matches.team2.unique()

winners = df_matches.winner.unique()

print(teams1)
print(teams2)
print(winners)


team_codes = { "team1" : {'Kolkata Knight Riders':0,'Chennai Super Kings':1,'Rajasthan Royals':2,
 'Mumbai Indians':3 ,'Deccan Chargers':4, 'Kings XI Punjab':5,
 'Royal Challengers Bangalore':6 ,'Delhi Daredevils':7 ,'Kochi Tuskers Kerala':8,
 'Pune Warriors':9, 'Sunrisers Hyderabad':10 ,'Rising Pune Supergiants':11,
 'Gujarat Lions':12},
               "team2" : {'Kolkata Knight Riders':0,'Chennai Super Kings':1,'Rajasthan Royals':2,
 'Mumbai Indians':3 ,'Deccan Chargers':4, 'Kings XI Punjab':5,
 'Royal Challengers Bangalore':6 ,'Delhi Daredevils':7 ,'Kochi Tuskers Kerala':8,
 'Pune Warriors':9, 'Sunrisers Hyderabad':10 ,'Rising Pune Supergiants':11,
 'Gujarat Lions':12},
               "toss_winner" : {'Kolkata Knight Riders':0,'Chennai Super Kings':1,'Rajasthan Royals':2,
 'Mumbai Indians':3 ,'Deccan Chargers':4, 'Kings XI Punjab':5,
 'Royal Challengers Bangalore':6 ,'Delhi Daredevils':7 ,'Kochi Tuskers Kerala':8,
 'Pune Warriors':9, 'Sunrisers Hyderabad':10 ,'Rising Pune Supergiants':11,
 'Gujarat Lions':12},
            "winner" : {'Kolkata Knight Riders':0,'Chennai Super Kings':1,'Rajasthan Royals':2,
 'Mumbai Indians':3 ,'Deccan Chargers':4, 'Kings XI Punjab':5,
 'Royal Challengers Bangalore':6,'Delhi Daredevils':7 ,'Kochi Tuskers Kerala':8,
 'Pune Warriors':9, 'Sunrisers Hyderabad':10 ,'Rising Pune Supergiants':11,
 'Gujarat Lions':12}
     
}

code_teams = ['Kolkata Knight Riders','Chennai Super Kings','Rajasthan Royals',
 'Mumbai Indians' ,'Deccan Chargers', 'Kings XI Punjab',
 'Royal Challengers Bangalore' ,'Delhi Daredevils' ,'Kochi Tuskers Kerala',
 'Pune Warriors', 'Sunrisers Hyderabad' ,'Rising Pune Supergiants',
 'Gujarat Lions']

df_matches.replace(team_codes,inplace=True)

print(df_matches.team1.unique())
print(df_matches.team1.unique())
print(df_matches.toss_winner.unique())
print(df_matches.winner.unique())

['Kolkata Knight Riders' 'Chennai Super Kings' 'Rajasthan Royals'
 'Mumbai Indians' 'Deccan Chargers' 'Kings XI Punjab'
 'Royal Challengers Bangalore' 'Delhi Daredevils' 'Kochi Tuskers Kerala'
 'Pune Warriors' 'Sunrisers Hyderabad' 'Rising Pune Supergiants'
 'Gujarat Lions']
['Royal Challengers Bangalore' 'Kings XI Punjab' 'Delhi Daredevils'
 'Kolkata Knight Riders' 'Rajasthan Royals' 'Mumbai Indians'
 'Chennai Super Kings' 'Deccan Chargers' 'Pune Warriors'
 'Kochi Tuskers Kerala' 'Sunrisers Hyderabad' 'Rising Pune Supergiants'
 'Gujarat Lions']
['Kolkata Knight Riders' 'Chennai Super Kings' 'Delhi Daredevils'
 'Royal Challengers Bangalore' 'Rajasthan Royals' 'Kings XI Punjab'
 'Deccan Chargers' 'Mumbai Indians' 'Pune Warriors' 'Kochi Tuskers Kerala'
 'Sunrisers Hyderabad' 'Rising Pune Supergiants' 'Gujarat Lions']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
[ 6  1  2  3  4  5  0  7  8  9 10 12 11]
[ 0  1  7  6  2  5  4  3  9  8 10 11 12]


In [9]:
# Now giving codes to Bat and field
toss_decision = df_matches.toss_decision.unique()
# 0 : field 1 : bat
decision_codes = { "toss_decision":{'field':0,'bat':1} }

df_matches.replace(decision_codes,inplace=True)

In [10]:
df_matches.head()

,id,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,venue
0,1,Bangalore,0,6,6,0,normal,0,0,140,0,M Chinnaswamy Stadium
1,2,Chandigarh,1,5,1,1,normal,0,1,33,0,"Punjab Cricket Association Stadium, Mohali"
2,3,Delhi,2,7,2,1,normal,0,7,0,9,Feroz Shah Kotla
3,4,Mumbai,3,6,3,1,normal,0,6,0,5,Wankhede Stadium
4,5,Kolkata,4,0,4,1,normal,0,0,0,5,Eden Gardens


In [11]:
# result,city and venue
result = df_matches.result.unique()
#  nommal : 0 , tie:1
print(result)

cities = df_matches.city.unique()

print(cities)

#venues = df_matches.venue.unique()

#print(venues)



['normal' 'tie']
['Bangalore' 'Chandigarh' 'Delhi' 'Mumbai' 'Kolkata' 'Jaipur' 'Hyderabad'
 'Chennai' 'Cape Town' 'Port Elizabeth' 'Durban' 'Centurion' 'East London'
 'Johannesburg' 'Kimberley' 'Bloemfontein' 'Ahmedabad' 'Cuttack' 'Nagpur'
 'Dharamsala' 'Kochi' 'Indore' 'Visakhapatnam' 'Pune' 'Raipur' 'Ranchi'
 'Abu Dhabi' 'Sharjah' 'Rajkot' 'Kanpur']


In [12]:
# chane city , venues , result
# Remove no results
encode_cities_venues_results = {
    "result" : {'normal':0,'tie':1,'no result':2},
    "venue" : {
        'M Chinnaswamy Stadium':0,'Punjab Cricket Association Stadium, Mohali':1,
 'Feroz Shah Kotla':2, 'Wankhede Stadium':3 ,'Eden Gardens':4,
 'Sawai Mansingh Stadium':5, 'Rajiv Gandhi International Stadium, Uppal':6,
 'MA Chidambaram Stadium, Chepauk':7, 'Dr DY Patil Sports Academy':8, 'Newlands':9,
 "St George's Park":10, 'Kingsmead':11, 'SuperSport Park':12, 'Buffalo Park':13,
 'New Wanderers Stadium':14, 'De Beers Diamond Oval':15, 'OUTsurance Oval':16,
 'Brabourne Stadium':17, 'Sardar Patel Stadium, Motera':18, 'Barabati Stadium':19,
 'Vidarbha Cricket Association Stadium, Jamtha':20,
 'Himachal Pradesh Cricket Association Stadium':21 ,'Nehru Stadium':22,
 'Holkar Cricket Stadium':23,
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium':24,
 'Subrata Roy Sahara Stadium':25,
 'Shaheed Veer Narayan Singh International Stadium':26,
 'JSCA International Stadium Complex':27, 'Sheikh Zayed Stadium':28,
 'Sharjah Cricket Stadium':29, 'Maharashtra Cricket Association Stadium':30,
 'Punjab Cricket Association IS Bindra Stadium, Mohali':31,
 'Saurashtra Cricket Association Stadium':32,'Green Park':33,
 'Dubai International Cricket Stadium':34
    },
    "city" : {
        'Bangalore':0, 'Chandigarh':1, 'Delhi':2, 'Mumbai':3, 'Kolkata':4, 'Jaipur':5, 'Hyderabad':6,
 'Chennai':7, 'Cape Town':8, 'Port Elizabeth':9, 'Durban':10, 'Centurion':11, 'East London':12,
 'Johannesburg':13, 'Kimberley':14, 'Bloemfontein':15, 'Ahmedabad':16, 'Cuttack':17, 'Nagpur':18,
 'Dharamsala':19, 'Kochi':20, 'Indore':21, 'Visakhapatnam':22, 'Pune':23, 'Raipur':24, 'Ranchi':25,
 'Abu Dhabi':26, 'Sharjah':27, 'Rajkot':28, 'Kanpur':29
    }
    
}
df_matches.replace(encode_cities_venues_results,inplace=True)

In [13]:
# write processed csv
df_matches.head()

,id,city,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,venue
0,1,0,0,6,6,0,0,0,0,140,0,0
1,2,1,1,5,1,1,0,0,1,33,0,1
2,3,2,2,7,2,1,0,0,7,0,9,2
3,4,3,3,6,3,1,0,0,6,0,5,3
4,5,4,4,0,4,1,0,0,0,0,5,4


In [14]:
# Get some visualizations
df_matches['winner'].hist(bins=50)

In [15]:
# Function that returns predictions

def classifier(X,Y):
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X,Y)
    return model

In [16]:
# Predictors : all the attributes used for prediction 
predictors = ['team1','team2','venue','city','toss_winner','toss_decision']
outcome = ['winner']

X = df_matches[predictors] # passing the list to select attributes used prediction
Y = df_matches[outcome]

# Train Model 
model = classifier(X,Y[outcome])

def getTeamCode(team):
    return team_codes["team1"][team]
def getVenueCode(venue):
    return encode_cities_venues_results["venue"][venue]
def getCityCode(city):
    return encode_cities_venues_results["city"][city]
def getDecisionCode(decision):
    return decision_codes["toss_decision"][decision]




C:\Users\Aniruddha\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [17]:
# Now call function that will predict winner
at = ['Chennai Super Kings','Rajasthan Royals','MA Chidambaram Stadium, Chepauk','Chennai','Rajasthan Royals','field']


team1 = getTeamCode(at[0])
team2 = getTeamCode(at[1])
venue = getVenueCode(at[2])
city = getCityCode(at[3])
toss_winner = getTeamCode(at[4])
toss_decision = getDecisionCode(at[5])

pass_attributes = []

pass_attributes.append(team1)
pass_attributes.append(team2)
pass_attributes.append(venue)
pass_attributes.append(city)
pass_attributes.append(toss_winner)
pass_attributes.append(toss_decision)


def predictWinner(model,attributes):
    prediction = model.predict([attributes])
    return prediction

prediction = predictWinner(model,pass_attributes)

def getTeamName(team_code):
    return code_teams[team_code]

print("Winning Team "+getTeamName(int(prediction[0])))

Winning Team Chennai Super Kings


In [18]:
# Cross Validation

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

test_model = SVC(gamma='scale')
test_model.fit(X_train,y_train[outcome])
test_predictions = test_model.predict(X_test)

test_accuracy = accuracy_score(test_predictions,y_test)

print("Test accuracy "+str(test_accuracy))


C:\Users\Aniruddha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TypeError: must be real number, not str